In [66]:
#Imports das bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns 
import os
import glob
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import MultiLabelBinarizer
import re

Visualizar dados de uma coluna do dataframe --> final[final['Origem_Aba'] == 'true_label']
Remover coluna de um dataframe --> final = final.drop(columns=['Unnamed: 0'])



In [67]:
#Normalizando o arquivo labels.xlsx
pyt1 = pd.read_excel('./vulnerable_files/labels/labels.xlsx', engine='openpyxl', sheet_name=None)
merged = []
special_tabs = ['codeql', 'bandit']

for tab_name, df in pyt1.items():

    if tab_name in special_tabs and 'label' in df.columns:
        df['Vulline'] = df['label']
        df = df.drop(columns=['label'])

    # Cria uma nova coluna com o nome da aba atual
    df['Sast_tool'] = tab_name
    merged.append(df)

df_pyt1 = pd.concat(merged, ignore_index=True)

df_pyt1 = df_pyt1.drop(columns=['Unnamed: 0'])


In [68]:
#Normalizando o arquivo PyTy2_final_labels.xlsx
pyt2 = pd.read_excel('./vulnerable_files/labels/PyTy2_final_labels.xlsx', engine='openpyxl')

df_pyt2 = pd.DataFrame()
df_pyt2['Filename'] = pyt2['Filename']
df_pyt2['Vulline'] = np.nan

colunas_sast = ['Mr. Chekideh', 'Semgrep', 'Bandit']

df_pyt2 = pyt2.melt(id_vars=['Filename'] ,value_vars=colunas_sast, var_name='Sast_tool', value_name='Vulline')

df_pyt2['Sast_tool'] = df_pyt2['Sast_tool'].replace("Mr. Chekideh", "true_label")


In [69]:
#Normalizando o arquivo siddiq_final_labelx.xlsx
siddiq = pd.read_excel('./vulnerable_files/labels/siddiq_final_labels.xlsx', engine='openpyxl')

df_siddiq = pd.DataFrame()
df_siddiq['Filename'] = pyt2['Filename']
df_siddiq['Vulline'] = np.nan

colunas_sast = ['Mr. Chekideh', 'Semgrep', 'Bandit']

df_siddiq = siddiq.melt(id_vars=['Filename'] ,value_vars=colunas_sast, var_name='Sast_tool', value_name='Vulline')

df_siddiq['Sast_tool'] = df_siddiq['Sast_tool'].replace("Mr. Chekideh", "true_label")


In [70]:
#Unificando todos oa dataframes em um único
df_final = pd.concat(objs=[df_pyt1, df_pyt2, df_siddiq])

In [71]:
# Definindo funções úteis

def unifyCweAndLine(df, columnName):
    df[columnName] = df.apply(lambda row: f'{row["line"]}({row["cwe"]})', axis=1)
    df = df.rename(columns={'filename':'Filename'})

    df_aggregated = (
        df.groupby("Filename")[columnName]
        .apply(lambda x: ",".join(x))
        .reset_index()
    )
    
    return df_aggregated

def remove_duplicates(cwe_list):
    if isinstance(cwe_list, float) and np.isnan(cwe_list):
        return []
    else:
        test = sorted(list(set(cwe_list.split(','))))
        # print(test)
        return test

def reder(col, df):
    temp = df[col][df[col].notna()].str.split(',').explode()
    preds = (temp.index.astype(str) + '-' + temp)
    preds = preds[preds.notna()].values
    return preds

def my_rec(labels, preds):
    rec_samples = [x for x in preds if x in labels]
    return len(rec_samples)/len(labels)

def my_pre(labels, preds):
    tp = [x for x in preds if x in labels]
    if len(preds) == 0:
        return 0
    return len(tp)/len([x for x in preds if '-nan' not in x])

def my_f1(precision, recall):
    return 2 * (precision * recall) / (precision + recall)

def bench_it(labels, preds, attack = ''):
    if attack != '':
        attack = '('+attack+')'
    labels = [x for x in labels if attack.lower() in x.lower()]
    attack.lower()
    preds = [x for x in preds if attack.lower() in x.lower()]
    
    pre = my_pre(labels, preds)
    rec = my_rec(labels, preds)
    # if attack == '':
    #     print(pre, 'labels:', labels, 'preds:', preds)

    
    #print(pre, labels, preds)

    if (pre + rec) == 0:
        return [pre, rec, 0]
    def f1(precision, recall):
        return 2 * (precision * recall) / (precision + recall)
    return [pre, rec, f1(pre, rec)]


def full_bench(preds, exploded_labels, dist_labels):
    labels_dist = exploded_labels.str.extract('\((.*)\)')[0].value_counts()  

    atts = list(labels_dist.index)
    atts.append('')
    sems = []
    for attack in atts:
        temp = bench_it(dist_labels, preds, attack)
        temp.extend([attack if attack != '' else 'all', labels_dist[attack] if attack != '' else sum(labels_dist)])
        sems.append(temp)
    df = pd.DataFrame(sems, columns = ['Precision', 'Recall', 'F1-Score', 'Attack', 'Support'])
    df = df.set_index('Attack')
    return df

def clear_data(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].apply(lambda x: re.sub(r'\b0(\d{1})\b|\b0(\d{2})\b', r'\1\2', x))
    df[column] = df[column].apply(lambda x: re.sub(r'(CWE-\d+)-0(\d)', r'\1-\2', x))
    df[column] = df[column].apply(lambda x: ','.join(remove_duplicates(x)))
    return df


<>:65: SyntaxWarning: "\(" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\("? A raw string is also an option.
<>:65: SyntaxWarning: "\(" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\("? A raw string is also an option.
C:\Users\levim\AppData\Local\Temp\ipykernel_19076\3668182045.py:65: SyntaxWarning: "\(" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\("? A raw string is also an option.
  labels_dist = exploded_labels.str.extract('\((.*)\)')[0].value_counts()


In [72]:
#Criando dataframe só com as ground truths e realizando limpeza

def remove_spaces(text):
    return re.sub(r'(\d) +\(', r'\1(', text)


df_ground_truth = df_final[df_final['Sast_tool'] == 'true_label']
df_ground_truth = df_ground_truth.copy()

df_ground_truth['Vulline'] = df_ground_truth['Vulline'].apply(lambda x: re.sub(r'\b0(\d{1})\b|\b0(\d{2})\b', r'\1\2', x))
df_ground_truth['Vulline'] = df_ground_truth['Vulline'].apply(remove_spaces)
df_ground_truth['Vulline'] = df_ground_truth['Vulline'].str.replace(', ', ',')
df_ground_truth['Vulline'] = df_ground_truth['Vulline'].apply(lambda x: ','.join(remove_duplicates(x)))


In [73]:
# Criando dataframe com ground truths e com resultados do bandit, e realizando limpeza dos dados
bandit = pd.read_json('C:/Users/levim/projetos/results/formatted_bandit.json')
semgrep = pd.read_json('C:/Users/levim/projetos/results/formatted_semgrep.json')

bandit_aggregated = unifyCweAndLine(bandit, 'bandit')
semgrep_aggregated = unifyCweAndLine(semgrep, 'semgrep')

df_bandit = df_ground_truth.merge(bandit_aggregated, on="Filename", how="left")
df_semgrep = df_ground_truth.merge(semgrep_aggregated, on="Filename", how="left")

df_bandit = clear_data(df_bandit, 'bandit')
df_semgrep = clear_data(df_semgrep, 'semgrep')

In [74]:
bandit_preds = reder('bandit', df_bandit)
semgrep_preds = reder('semgrep', df_semgrep)

exploded_labels = df_bandit['Vulline'].str.split(',').explode()
dist_labels = (exploded_labels.index.astype(str) + '-' + exploded_labels).values

results_bandit = bench_it(dist_labels, bandit_preds)
results_semgrep = bench_it(dist_labels, semgrep_preds)

data = {
    'Tool': ['Bandit', 'Semgrep'],
    'Precision': [results_bandit[0], results_semgrep[0]],
    'Recall': [results_bandit[1], results_semgrep[1]],
    'F1-Score': [results_bandit[2], results_semgrep[2]]
}

final_result = pd.DataFrame(data)
final_result

,Tool,Precision,Recall,F1-Score
0,Bandit,0.288462,0.057803,0.096308
1,Semgrep,0.677778,0.117534,0.200328
